In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py
!pip install GLU
!pip install gym==0.22
!apt install ffmpeg
!pip install setuptools==65.5.0 "wheel<0.40.0"
!pip install stable-baselines3[extra]==1.6.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.2 MB/s eta 0:00:00
  Created wheel for nes_py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=535718 sha256=b26e8696c9e5fbaefe36fd1b1e04b92ecb1139383b3cfc9f0e9263f9d34a48cd
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes_py
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.8/371.8 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GLU: filename=glu-0.0.18-py3-none-any.whl size=6456 sha256=5c64fd858e18c002825e83bc8357300ec07102cb34e18469e06670ddfc47

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 48.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616797 sha256=d5aeecc2f953d9169f0537ddb7565ba28e6f42fbe803b243c97029806c6b93f0
  Stored in directory: /root/.cache/pip/wheels/81/aa/90/b67df76370d3916a2189b662cf48da38ce41a4e7e58b6abff5
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=5f5fa84f63f8a931f75a0d4f8cac118bb4f26c8fe1

In [ ]:

import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
from gym import wrappers
import io
import base64
from IPython.display import HTML
import gym
from gym import wrappers
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt
import stable_baselines3
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
MY_ACTIONS = [["right"], ["right", "A"]]

Eviroment for downscaled version (this env is what i used for my models)

In [ ]:
# 1. Create the base environment
env = gym.make('SuperMarioBros-v0')
# 2. Simplify the controls
env = JoypadSpace(env, MY_ACTIONS)

# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)


from gym_utils import SMBRamWrapper
# Setup cropping size
x0 = 0
x1 = 16
y0 = 0
y1 = 13
n_stack = 1
n_skip = 4

env = SMBRamWrapper(env, [x0, x1, y0, y1], n_stack=n_stack, n_skip=n_skip)


# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 2, channels_order='last')

Normal enviroment just with gray scale (just run this if you using or want to train a model in this env, its much slower because there is no downscale)

In [ ]:
# 1. Create the base environment
env = gym.make('SuperMarioBros-v0')
# 2. Simplify the controls
env = JoypadSpace(env, MY_ACTIONS)

# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)

# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

Continuation

In [ ]:
state = env.reset()

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.0001,
            n_steps=512)




Using cpu device


Training cells, just run it if you want to train a new model

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_1


/usr/local/lib/python3.10/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


A saída de streaming foi truncada nas últimas 5000 linhas.
|    time_elapsed         | 5867         |
|    total_timesteps      | 858624       |
| train/                  |              |
|    approx_kl            | 6.421481e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.13        |
|    explained_variance   | 0.369        |
|    learning_rate        | 0.0001       |
|    loss                 | 21.5         |
|    n_updates            | 16760        |
|    policy_gradient_loss | -0.000106    |
|    value_loss           | 284          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 146          |
|    iterations           | 1678         |
|    time_elapsed         | 5870         |
|    total_timesteps      | 859136       |
| train/                  |              |
|    approx_kl            | 0.00716284

In [ ]:
model_name ="best_model_950000" # name that you want to save the model

model.save(model_name)


Cells to load and test the model

In [ ]:
model_name ="best_model_950000" #put there the model name or directory/name

In [ ]:
# Load model
model = PPO.load(model_name)

NameError: name 'PPO' is not defined

In [ ]:
state = env.reset()

In [ ]:
#thsi cell creates a video in the output_model folder of the model playing

env = stable_baselines3.common.vec_env.VecVideoRecorder(env, './output_model',
                       record_video_trigger=lambda x: x == 0, video_length=10000,
                       name_prefix="model_video-{}".format('SuperMarioBros-v0'))

env.reset()
for _ in range(10000 + 1):
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)

env.close()

/usr/local/lib/python3.10/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Saving video to /content/output_model/100000_step_model_sample-SuperMarioBros-v0-step-0-to-step-100000.mp4
